# DM Simulations

The array parameters will contain all the parameters extracted from inifile.dat

* parameters[0] is the dimension D of the system (int type)

* parameters[1] is the length of the box L in the dimensionless units of the code, it is a square box (float type)

* parameters[2] is the number of sites by length N (int type)

* parameters[3] is the total mass M, in dimensionless units of the system (float type)

* parameters[4] is name of the field configuration to be simulated, (string type). The available fields are:
    * spin0
    * spin2

* parameters[5] is the condensate fraction (float type). If the field is different than spin0, the condensate fraction is automatically 1.

* parameters[6] is an array defining the initial conditions of the field.
    * parameters[6][0][0] contains the type of initial condition for the field or condensate (string type). The available conditions are
        * testing
        * random
        * centralsoliton
        * multiple-solitons
        * from-file
        
      For "centralsoliton" the array contains one parameter: radc (float type) stored in parameters[6][0][1].
      
      For "multiple-solitons" the array contains two parameters: radc (float type) stored in parameters[6][0][1] and numbersolitons (int type) stored in parameters[6][0][2].
      
      For "from-file" the initial file is stored in parameters[6][0][1].
      
    * parameters[6][1][0] contains the type of initial condition for the particles (string type) if we choose spin0 and the condensate fraction is less than 1. The available conditions are
        * testing
        * random
        * maxwellian
        * from-file
        
        For "maxwellian" the array contains one parameter: sigma (float type) stored in parameters[6][0][1].
        
* parameters[7] contains information about the particles.
    * parameters[7][0] contains the number of test particles (int type) 
    * parameters[7][1] contains the type of particle-mesh density assignment for particles. The available types are:
        * nearestgridpoint
        * cloudincell
    
    If the type of field is not spin0 or condensate fraction is 1, these values are assigned to be 0 and nearestgridpoint by default, since that part of the array won't be used.
    
* parameters[8] is an array with the information about the type of potential considered.
    * parameters[8][0] gives the name of the potential (string type). We have:
        * testing
        * full
        
        Testing is a potential well, storing in parameters[8][1] the length of the well (float type).
        
        Full contains the gravitational, harmonical and self-interaction. In parameters[8][1] we indicate if the gravitational potential is on or off (True or False). In parameters[8][2] we store the harmonic frequency omega (float type), and in parameters[8][3] we have the value of the self-coupling g (float type).

* parameters[9] is an array indicating the type of evolver.
    * parameters[9][0] gives the name of the evolver (string type). We have:
        * kickdriftkick
        * driftkickdrift
    * parameter[9][1] gives the number of iterations of the simulation (int type)
    * parameter[9][2] gives the number of steps to save the field in the output field (int type)
    * parameters[9][3] indicates if the propagation is in imaginary time or not (True or False)
    
* parameters[10] is the folder where the ouptut files will be saved (string type)

In [1]:
#Load the modules needed for the program
import time
import numpy as np
from utilities import storer, grid, fields, potentials, evolvers

#Defines the main function
def main(file):
    #Initialize the counting of time execution
    start_time = time.time()

    #Load parameters
    parameters=storer.reader(file)

    #Create the grids
    grids=grid.creategrid(parameters)

    #Create the initial configuration of the fields and generate the potentials
    if parameters[4]=='spin0':
        psi=fields.spin0(parameters, grids)
        pot=potentials.potential(parameters, grids, psi,1)
        potnc=potentials.potential(parameters, grids, psi,0)
    if parameters[4]=='spin2':
        psi=fields.spin2(parameters, grids)
        pot=potentials.potential(parameters, grids, psi,1)
        potnc=pot

    #Evolve the system
    if parameters[9][0]=='kickdriftkick' or parameters[9][0]=='driftkickdrift':
        evolvedarray=evolvers.kickdrift(parameters, grids, psi, pot, potnc, 1e-2)

    print(" ")
    print("--- %s seconds ---" % (time.time() - start_time))

In [2]:
#Reference quantities
mboson=1e-22*1.78e-36
g=1e-87
Mreference=6.658e37
dim=3
rreference=((1.054e-34)**2/(6.67e-11*Mreference*mboson**2))**(1/(4-dim))
treference=(mboson*rreference**2)/(1.054e-34)
vreference=rreference/treference
gdimensionless=(Mreference*treference**2*g)/(mboson**2*rreference**5)
     
print(f'Reference scales')
print(f'r_ref= {rreference*3.24e-20} kpc')
print(f't_ref= {treference*3.17e-17} Gyr')
print(f'v_ref= {vreference} m/s')
print(f'dimensionless g= {gdimensionless}')

Reference scales
r_ref= 2.5580946098545043 kpc
t_ref= 0.33371969534861545 Gyr
v_ref= 7499.788500948769 m/s
dimensionless g= 0.07590859911982109


In [3]:
#Initialize the program
main("inifile.dat")

*-----------------------------------*
Dimension: 2
Box length: 40.0
N: 64
Total mass: 1.0
 
Field configuration: spin0
Condensate fraction: 0.8
Initial configuration condensate: centralsoliton
  radc: 2.0
Initial configuration particles: random
Number of test particles: 1000
Density assignment: cloudincell
 
Potential type: full
Potential parameters:
  gravitational: True
  omega: 0.0
  selfcoupling: 0.0
 
Evolver: kickdriftkick
Number of iterations: 30000
Saving every: 100 steps
Imaginary time propagation: False
 
Output files in: outputfiles/testing/
*-----------------------------------*
************************************************************|100% 
--- 163.1563949584961 seconds ---
